In [1]:
import numpy as np
from sympy import *

x, y= symbols("x y")
f = x + y
diff(f, x)

1

In [2]:
x, y= symbols("x y")
f = 2*x + 3*y + 5
diff(f, x)

2

In [3]:
x, w= symbols("x w")
a = [[x**2*w, 3*x*w**2], [2*x*w**3 ,6*x**3*w**(-1)]]
list1 = []
list2 = []
[[list1.append(diff(i, w)) for i in j] for j in a]
[[list2.append(diff(i, x)) for i in j] for j in a]
print('w로 미분:',list1, '\nx로 미분:',list2)

w로 미분: [x**2, 6*w*x, 6*w**2*x, -6*x**3/w**2] 
x로 미분: [2*w*x, 3*w**2, 2*w**3, 18*x**2/w]


## Back Propagation (역전파)

<img src='images/1.jpg' />

<img src='images/2.jpg' />

<img src='images/3.jpg' />

## 곱셈 계층

In [4]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다
        dy = dout * self.x
        return dx, dy

In [5]:
apple = 100
apple_num = 2
tax = 1.1
# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()
# forward(순전파)
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
price # 220

220.00000000000003

## 덧셈 계층

In [6]:
class AddLayer:
    def __init__(self):
        pass # 더하기는 미분하면 1이니까 저장할 필요 없음
    def forward(self, x, y):
        out = x + y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1
# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward(순전파)
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num) # 여기 채우기
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax) # 여기 수정
print('forward:', price) # 220

# backward(역전파)
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dorange_price)
print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax) # 110 2.2 3.3 165 650

forward: 715.0000000000001
715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


### 활성화 함수

<img src='images/활성화함수계산.jpg' />

In [8]:
class Relu:
    def __init__(self):
        self.mask = None
    def forward(f, x):
        self.mask = (x <= 0)
        out = x.copy() # 깊은 복사(deep copy), shallow copy(얕은 복사) => 주소 복사
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [9]:
import numpy as np

In [10]:
# 참고
x = np.array([[1,2],[3,-2]])
x[1][x[1]<0] # fancy index

array([-2])

In [11]:
# fancy index 연습
x_30 = np.array([[20,50,70],[80,-7,20]])
# fancy index로 30보다 큰 데이터 추출
print('fancy index:', x_30[x_30>30])
# 일반 반복문이나 slicing을 해서 추출
[[ i for i in j if i>30 ] for j in x_30 ]

fancy index: [50 70 80]


[[50, 70], [80]]

In [12]:
# T
# x_30 = np.array([[20,50,70],[80,-7,20],[100,20,-100,200,50]]) # numpy라서..
x_30 = np.array([[20,50,70,0,0],[80,-7,20,0,0],[100,20,-100,200,50]])
# [ i for i in x_30 if i>30 ] # 데이터의 갯수가 맞지 않아서 오류
# p = zip(*[[ i for i in x_30[j] if i>30 ] for j in range(len(x_30)) ])
# for i in p:
#     print(i)
[[ i for i in x_30[j] if i>30 ] for j in range(len(x_30)) ]

[[50, 70], [80], [100, 200, 50]]

In [13]:
x = np.array([[1.0, -0.5],[-2.0, 3.0]])
x

array([[ 1. , -0.5],
       [-2. ,  3. ]])

In [14]:
mask = (x < 0)
mask

array([[False,  True],
       [ True, False]])

## 시그모이드 계층

$$ y = \frac{1}{1 + exp(-x)} $$
#### 1단계)
'/' 노드, 즉 $y = \frac{1}{x}$을 미분하면
$$ \frac{\partial y}{\partial x} = - \frac{1}{x^2} = -y^2 $$

역전파 때는 상류에서 홀더 들어온 값에 $ -y^2 $ (순전파의 출력을 제곱한 후 마이너스를 붙인 값)을 곱해서 하류로 전달함
<img src='images/sigmoid_1.jpg' />

#### 2단계)
'+' 노드는 상류의 값을 여과없이 하류로 내보내는 것이 다이다.
<img src='images/sigmoid_2.jpg' />

#### 3단계)
'exp' 노드는 y = exp(x) 연산을 수행한다. <br>계산 그래프에서는 상류의 값에 순전파 때의 출력을 곱해 하류로 전달
<img src='images/sigmoid_3.jpg' />

#### 4단계)
'x' 노드는 순전파 때의 값을 서로 바꿔서 곱한다.
<img src='images/sigmoid_4.jpg' />

<img src='images/sigmoid_total.jpg' />

In [15]:
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out # 결국
        # back에서 전달된 값 dout에 
        # sigmoid 함수의 미분을 sigmoid * (1-sigmid)를 곱한 결과
        return dx # 반환

### Affine / Softmax 계층 구현하기
#### Affine 계층
신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 내적(넘파이에서는 np.dot()을 사용) <br>
여기서는 X, W, B는 각각의 형상이 (2,),(2,3),(3,)인 다차원 배열 <br>
그러면 뉴런의 가중치의 합은 Y

In [16]:
X = np.random.rand(2) # 입력
W = np.random.rand(2, 3) # 가중치
B = np.random.rand(3) # 편향(bias)
print(X.shape, W.shape, B.shape)
Y = np.dot(X, W) + B
Y

(2,) (2, 3) (3,)


array([0.63962619, 0.26274779, 1.27579088])

In [17]:
class Affine:
    def __init__(self):
        self.W = W # 구하고자 하는 것은 W, b
        self.b = b
        self.x = None # 입력
        self.dW = None # 업데이트 하기 위해 미분(w에 대한)
        self.db = None # 업데이트 하기 위해 미분(b에 대한)
    def forward(self, x):
        self.x = x # 입력데이터를 멤버변수 x에 저장
        out = np.dot(x, self.W) + self.b # 입력데이터와 W의 행렬곱과 bias(벡터)의 더하기
        return out
    def backward(f, dout):
        dx = np.dot(dout, self.W.T) # 마지막에서부터의 오차(dout)과 W행렬의 traspost의 곱
  
        # 곱한 결과(dx)의 transpose한 것과 backward의 
        # 파라미터 입력으로 전달되는 dout을 곱한 결과를 dW에 저장
        # a * delta
        self.dW = np.dot(self.x.T, dout) 
        self.db = np.sum(dout, axis=0) # bias는 총합(delta)
        return dx

<img src='images/back.jpg' />

<img src='images/back1_1019.jpg' />

### Softmax-with-Loss 계층구현

In [18]:
from common.functions import *
import sys, os
sys.path.append(os.pardir)

In [21]:
class SoftMaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None # Softmax의 출력
        self.x = None # 정답 레이블(one hot 벡터)
    def forward(self, x, t):
        self.t = f
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

## 오차 역전파 법 구하기

In [23]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [28]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftMaxWithLoss()
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    # x: 입력데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t=np.argmax(t, axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.parmas['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.parmas['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.parmas['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.parmas['b2'])
        return grads
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        # 역전파(backward)
        dout = t
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse() # backward 하기 위해 순서를 뒤집음
        for layer in layers:
            dout = layer.backward(dout)
        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads

In [26]:
from dataset.mnist import load_mnist

In [31]:
x_test = np.array([[11,22,33,44,55],[1,2,3,4,5]])
x_test.shape

(2, 5)

In [33]:
x_test

array([[11, 22, 33, 44, 55],
       [ 1,  2,  3,  4,  5]])

In [38]:
x_test.reshape(5, -1)

array([[11, 22],
       [33, 44],
       [55,  1],
       [ 2,  3],
       [ 4,  5]])

In [44]:
x_zeros = np.zeros((3,4,5))
print(x_zeros.ndim) # n차원 확인
# 뒤에가 -1이면 알아서 맞춰줌, 3차원을 2차원으로 만들어줌?
reshaped_ = x_zeros.reshape(20, -1) 
reshaped_.ndim

3


2

In [52]:
x_zeros = np.zeros((3,4,5,10))
print(x_zeros.shape)
reshaped_ = x_zeros.reshape((5,4,-1)) 
reshaped_.shape

(3, 4, 5, 10)


(5, 4, 30)

In [53]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

TypeError: Affine.__init__() takes 1 positional argument but 3 were given

<img src='images/시그모이드.jpg' />

<img src='images/시그모이드2.jpg' />

<img src='images/계산그래프.jpg' />

합성함수
$$ z(x) = 1 + x $$

$$ a(x) = 1 + e^{x} $$

$$ a(z(x)) = ? $$

$$ a(z(x)) = 1 + e^{1 + x} $$

<div style="border: 1px solid black; width: 30%; float: right;">
$$ (log x)\acute{} = \frac{1}{x} $$ <br>
$$ (e^{x})\acute{} = e^{x} $$ <br>
$$ (\cos{x})\acute{} = -\sin{x} $$ <br>
$$ (\sin{x})\acute{} = \cos{x} $$
</div>
<br>

1) $ e^{x^{2} + 2x + 3} $ x로 미분
--- 
2) $ (e^{t} + 3t + 9)^{3} $ t로 미분
--- 
3) $ \log(\cos\theta + \sin\theta + 6\theta^{2} + 9) \theta $ 로 미분 
--- 
4) $ e^{x^{2} + 6x + 3} $ x로 미분